In [1]:
import requests
import pandas as pd
from typing import Optional, Dict, List, Any
import logging

In [2]:
# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
class SportsDataProcessor41:
    BASE_URL = "https://st-fn-cdn001.akamaized.net/olimp/ru/Europe:Berlin/gismo"
    HEADERS = {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
        "cache-control": "no-cache",
        "origin": "https://st-cdn001.akamaized.net",
        "pragma": "no-cache",
        "referer": "https://st-cdn001.akamaized.net",
        "sec-ch-ua": '"Microsoft Edge";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "cross-site",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"
    }

    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update(self.HEADERS)

    def fetch_data(self, url_suffix: str) -> Optional[Dict]:
        """Perform a GET request and return JSON data."""
        try:
            response = self.session.get(f"{self.BASE_URL}{url_suffix}")
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            logging.error(f"Request error: {e}")
            return None

    def _get_nested_data(self, data: Dict, keys: List[str]) -> Any:
        """Recursively retrieve nested data from a dictionary."""
        try:
            for key in keys:
                if isinstance(data, dict) and key in data:
                    data = data[key]
                elif isinstance(data, list) and isinstance(key, int) and 0 <= key < len(data):
                    data = data[key]
                else:
                    return None
            return data
        except (KeyError, TypeError, IndexError):
            logging.error("Error retrieving nested data.")
            return None

    def create_dataframe(self, data: Dict, path: List[str], columns_to_drop: List[str] = None) -> Optional[pd.DataFrame]:
        """Create a DataFrame from JSON data at the specified path."""
        result = self._get_nested_data(data, ['doc', 0, 'data'] + path)
        if not result:
            return None
        df = pd.json_normalize(result)
        if columns_to_drop:
            df.drop(columns=columns_to_drop, errors='ignore', inplace=True)
        return df

    def process_sports_hierarchy(self) -> Optional[pd.DataFrame]:
        """Process sports hierarchy data."""
        data = self.fetch_data("/config_sports/41/0/")
        if not data:
            return None

        df = self.create_dataframe(data, path=[], columns_to_drop=['_sk', '_doc'])
        if df is not None:
            df['urls0'] = "/config_tree_mini/41/0/" + df['_sid'].astype(str)
        return df

    def process_second_level(self, url_suffix: str) -> Optional[pd.DataFrame]:
        """Process second-level data."""
        data = self.fetch_data(url_suffix)
        if not data:
            return None

        df = self.create_dataframe(
            data,
            path=[0, 'realcategories'],
            columns_to_drop=['_sk', '_doc', 'uniquetournaments', 'tournaments']
        )
        if df is not None:
            df['url1'] = "/category_get/" + df['_rcid'].astype(str)
            df['url2'] = f"{url_suffix}/" + df['_rcid'].astype(str)
            self._expand_cc_column(df)
            if '_isk' in df.columns:
                df = df[df['_isk'] != 1000]
        return df

    def process_third_level(self, url_suffix: str) -> Optional[pd.DataFrame]:
        """Process third-level data."""
        data = self.fetch_data(url_suffix)
        if not data:
            return None

        return self.create_dataframe(
            data,
            path=[0, 'realcategories', 0, 'tournaments'],
            columns_to_drop=['_sk', '_doc']
        )

    @staticmethod
    def _expand_cc_column(df: pd.DataFrame) -> None:
        """Expand the 'cc' column in a DataFrame."""
        if 'cc' in df.columns:
            expanded_cc = pd.json_normalize(df['cc']).add_prefix('cc_').fillna('None')
            df.drop(columns=['cc'], inplace=True)
            df = pd.concat([df, expanded_cc], axis=1)
            
    def full_pipeline(self) -> Dict[str, Any]:
        """Full pipeline to process all data."""
        results = {}

        frame0 = self.process_sports_hierarchy()
        if frame0 is None:
            return results
        results['Frame'] = frame0

        second = {}
        if not frame0.empty:
            for i, url in enumerate(frame0['urls0']):
                frame1_key = f"Frame1_{i}"
                frame1 = self.process_second_level(url)
                if frame1 is not None:
                    second[frame1_key] = frame1
            results['Frame1'] = second
            frame1 = results['Frame1']['Frame1_0']

            third = {}
            if not frame1.empty and 'url2' in frame1.columns:
                for i, url in enumerate(frame1['url2'][:11]):
                    key_f = f"{frame1['name'][i]}"
                    logging.info(f"Processing {key_f}")
                    frame2 = self.process_third_level(url)
                    if frame2 is not None:
                        quant = {}
                        if not frame2.empty and 'currentseason' in frame2.columns:
                            frame2 = frame2[~frame2['currentseason'].duplicated()].reset_index(drop=True)
                            third[key_f] = [frame2,]
                                
        return results, third

In [4]:
processor = SportsDataProcessor41()
results, th = processor.full_pipeline()

2025-04-08 09:46:35,209 - INFO - Processing Международный
2025-04-08 09:46:35,422 - INFO - Processing Международные матчи (клубы)
2025-04-08 09:46:35,601 - INFO - Processing Англия
2025-04-08 09:46:35,817 - INFO - Processing Германия
2025-04-08 09:46:36,078 - INFO - Processing Испания
2025-04-08 09:46:36,276 - INFO - Processing Италия
2025-04-08 09:46:36,458 - INFO - Processing Франция
2025-04-08 09:46:36,626 - INFO - Processing Австралия
2025-04-08 09:46:36,879 - INFO - Processing Австрия
2025-04-08 09:46:37,088 - INFO - Processing Австрия любители
2025-04-08 09:46:37,293 - INFO - Processing Азербайджан


## TEST

In [14]:
results['Frame1']['Frame1_0']

,_id,_sid,_rcid,name,cc._doc,cc._id,cc.a2,cc.name,cc.a3,cc.ioc,cc.continentid,cc.continent,cc.population,url1,url2
0,4,1,4,Международный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/category_get/4,/config_tree_mini/41/0/1/4
1,393,1,393,Международные матчи (клубы),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/category_get/393,/config_tree_mini/41/0/1/393
2,1,1,1,Англия,countrycode,240.0,en,Англия,АНГ,ENG,1.0,Европа,51500000.0,/category_get/1,/config_tree_mini/41/0/1/1
3,30,1,30,Германия,countrycode,80.0,de,Германия,ГЕР,GER,1.0,Европа,82500000.0,/category_get/30,/config_tree_mini/41/0/1/30
4,32,1,32,Испания,countrycode,199.0,es,Испания,ИСП,ESP,1.0,Европа,46000000.0,/category_get/32,/config_tree_mini/41/0/1/32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,92,1,92,Эстония,countrycode,67.0,ee,Эстония,ЭСТ,EST,1.0,Европа,1315635.0,/category_get/92,/config_tree_mini/41/0/1/92
133,914,1,914,Эфиопия,countrycode,68.0,et,Эфиопия,ЭФИ,ETH,4.0,Африка,102403000.0,/category_get/914,/config_tree_mini/41/0/1/914
134,322,1,322,Южная Африка,countrycode,197.0,za,Южная Африка,ЗАФ,RSA,4.0,Африка,49300000.0,/category_get/322,/config_tree_mini/41/0/1/322
135,502,1,502,Ямайка,countrycode,106.0,jm,Ямайка,ЯМА,JAM,2.0,Северная Америка,2900000.0,/category_get/502,/config_tree_mini/41/0/1/502


In [20]:
th['Международный'][0]

,_id,_sid,_rcid,_isk,_tid,_utid,seasonid,currentseason,name,roundbyround
0,2307,1,4,1,2307,436,105587.0,105587,"Олимпийские игры, группа А",True
1,6996,1,4,5,6996,2515,NaN,111974,Квалификация Олимпийсих игр (женщины),False
2,147109,1,4,5,147109,437,105589.0,105589,"Олимпийские игры, Женщины, Группа A",True
3,3948,1,4,10,3948,16,101177.0,101177,"Кубок Мира, Плей-офф",True
4,1757,1,4,20,1757,11,127075.0,127075,"Кубок Мира Квалификация, УЕФА, Группа А",True
5,6922,1,4,39,6922,308,108589.0,108589,"Кубок Мира Квалификация, АФК, Раунд 1",True
6,6692,1,4,51,6692,309,10874.0,121357,"Кубок Мира Квалификация, ОФК 1ый раунд",True
7,7271,1,4,62,7271,14,115355.0,115355,"Кубок Мира Квалификация, КОНКАКАФ 2ый раунд, Г...",True
8,1434,1,4,77,1434,295,109025.0,109025,"Кубок Мира Квалификация, КОНМЕБОЛ",True
9,9360,1,4,100,9360,13,112442.0,112442,"FIFA World Cup, Qualification CAF, Group A",False


## Second Part

In [70]:
class SportsDataProcessor(SportsDataProcessor41):
        def process_league_summary(self, season_id: int) -> Optional[pd.DataFrame]:
            """Process league summary statistics."""
            url = f"/stats_season_leaguesummary/{season_id}/main"
            data = self.fetch_data(url)
            if not data:
                return None
    
            try:
                league_data = self._get_nested_data(data, ['doc', 0, 'data'])
                df = pd.json_normalize(league_data)
                return df.add_prefix('league_').drop(columns=['league__doc'], errors='ignore')
            except (KeyError, TypeError) as e:
                print(f"League summary error: {e}")
                return None
    
        def process_season_goals(self, season_id: int) -> Optional[pd.DataFrame]:
            """Process goals statistics."""
            url = f"/stats_season_goals/{season_id}/main"
            data = self.fetch_data(url)
            if not data:
                return None
    
            try:
                teams_data = self._get_nested_data(data, ['doc', 0, 'data', 'teams'])
                total_data = self._get_nested_data(data, ['doc', 0, 'data', 'totals'])
                if not teams_data or not total_data:
                    return pd.DataFrame()
                    
                sk = pd.json_normalize(total_data).add_prefix('totals_')
                df = pd.json_normalize(teams_data)
                sk = pd.concat([sk]*len(df), ignore_index=True)
                if sk.empty or df.empty:
                    return pd.DataFrame()
                return pd.concat([df, sk], axis=1).drop(columns=['team._doc', 'team._id', 'team._sid', 'team.virtual', 'team.haslogo', 'team.iscountry', 'team.nickname'],axis=1,  errors='ignore')
            except (KeyError, TypeError) as e:
                print(f"Season goals error: {e}")
                return None
        def process_form_tables(self, season_id: int) -> Optional[pd.DataFrame]:
            """Process form tables."""
            data = self.fetch_data(f"/stats_formtable/{season_id}")
            if not data:
                return pd.DataFrame()
        
            try:
                formrows = self._get_nested_data(data, ['doc', 0, 'data', 'teams'])
        
                if not formrows:
                    print("⚠️ formrows пуст, возвращаю пустой DataFrame")
                    return pd.DataFrame()
        
                df = pd.DataFrame(formrows)
        
                # Безопасно нормализуем нужные колонки
                cols_to_expand = ['form', 'nextopponent', 'position', 'team']
                for col in cols_to_expand:
                    if col in df.columns:
                        sample = df[col].iloc[0]
                        try:
                            if isinstance(sample, list):
                                if all(isinstance(item, dict) for item in sample):
                                    # список словарей
                                    expanded = pd.json_normalize(sample).add_prefix(f"{col}_")
                                    expanded = pd.DataFrame([expanded.iloc[0]] * len(df))
                                else:
                                    # список обычных значений
                                    expanded = pd.DataFrame(df[col].tolist()).add_prefix(f"{col}_")
                            elif isinstance(sample, dict):
                                expanded = pd.json_normalize(df[col]).add_prefix(f"{col}_")
                            else:
                                print(f"⚠️ Колонка '{col}' содержит неподдерживаемый тип: {type(sample)}, пропускаю")
                                continue
                            df = pd.concat([df.drop(columns=[col]), expanded], axis=1)
                        except Exception as e:
                            print(f"❌ Ошибка при обработке колонки '{col}': {e}")
        
                # Удаляем "мусорные" и ненужные поля
                drop_columns = [
                    'team._doc', 'team.virtual', 'team.nickname', 'team.iscountry', 'team.haslogo',
                    'played.totalhome', 'played.totalaway', 'played.home', 'played.away',
                    'win.totalhome', 'win.totalaway', 
                    'draw.totalhome', 'draw.totalaway', 
                    'loss.totalhome', 'loss.totalaway',
                    'goalsfor.totalhome', 'goalsfor.totalaway',
                    'goalsagainst.totalhome', 'goalsagainst.totalaway',
                    'goaldifference.totalhome', 'goaldifference.totalaway',
                    'points.totalhome', 'points.totalaway',
                    'nextopponent.team._doc', 'nextopponent.team._id', 'nextopponent.team._sid', 
                    'nextopponent.team.virtual', 'nextopponent.team.mediumname', 'nextopponent.team.abbr',
                    'nextopponent.team.nickname', 'nextopponent.team.iscountry', 'nextopponent.team.haslogo',
                    'nextopponent.date._doc', 'nextopponent.date.tz', 'nextopponent.date.tzoffset'
                ]
        
                df.drop(columns=drop_columns, errors='ignore', inplace=True)
                return df
        
            except (KeyError, TypeError, ValueError) as e:
                print(f"❌ Form tables error: {e}")
                return None

                
        def process_season_meta(self, season_id: int) -> Optional[pd.DataFrame]:
            """Process season metadata."""
            data = self.fetch_data(f"/stats_season_meta/{season_id}")
            if not data:
                return None
    
            try:
                season_data = self._get_nested_data(data, ['doc', 0, 'data', 'season'])
                cc_data = self._get_nested_data(data, ['doc', 0, 'data', 'realcategory', 'cc'])
                # Проверяем, является ли cc_data списком или словарем
                cc_df = pd.DataFrame()  # Инициализируем пустой DataFrame
                if isinstance(cc_data, (list, dict)):
                    cc_df = pd.json_normalize(cc_data).add_prefix('cc_')
                    
                dfs = [
                    pd.json_normalize(season_data['start']).add_prefix('start_'),
                    pd.json_normalize(season_data['h2hdefault']).add_prefix('h2h_'),
                    pd.json_normalize(season_data['end']).add_prefix('end_')
                ]
                # Проверяем, что cc_data не None, перед добавлением
                if cc_data is not None:
                    dfs.append(pd.json_normalize(cc_data).add_prefix('cc_'))
    
                return pd.concat(dfs, axis=1).drop(columns=['h2h_matchid', 'h2h_teamidhome', 'h2h_teamidaway', 'start_time', 'start_date', 'start_tzoffset', 'end_time', 
                                                            'end_date', 'end_tzoffset', 'cc__id', 'cc_a2', 'cc_a3', 'cc_ioc', 'start__doc', 'start_tz', 'end__doc', 'end_tz',
                                                            'cc__doc'], axis=1, errors='ignore')
            except (KeyError, TypeError) as e:
                print(f"Season meta error: {e}")
                return None
        def process_season_tables(self, season_id: int) -> Optional[pd.DataFrame]:
            """Process season tables."""
            data = self.fetch_data(f"/stats_season_tables/{season_id}")
            if not data:
                return None
    
            try:
                tablerows = self._get_nested_data(data, ['doc', 0, 'data', 'tables', 0, 'tablerows'])
                if not isinstance(tablerows, list):
                    print(f"tablerows не список для сезона {season_id}. tablerows = {tablerows}")
                    return None

                df = pd.json_normalize(tablerows)
    
                for col in ['promotion', 'team']:
                    if col in df.columns:
                        expanded = pd.json_normalize(df[col]).add_prefix(f"{col}_")
                        df = pd.concat([df.drop(columns=[col]), expanded], axis=1)
    
                return df.drop(['_doc','promotion._id','promotion.cssclass', 'promotion.position', 'team._doc', 'team._sid','team.nickname', 'team.iscountry','changeTotal', 'changeHome', 'changeAway',  # Изменения позиций за сезон не так важны
                                'sortPositionTotal', 'sortPositionHome', 'sortPositionAway',  # Дублирует pos*
                                'team._id','team.virtual', 'team.haslogo',  # Технические ID, логотип не нужен
                                'team.mediumname', 'team.abbr',  # Название дублируется в `team.name`
                                'promotion._doc', 'promotion.shortname'  # Достаточно `promotion.name` 
                               ], axis=1, errors='ignore').infer_objects(copy=False).fillna(0)
            except (KeyError, TypeError) as e:
                print(f"Season tables error: {e}")
                return None               
        def process_season_data(self, season_id: int) -> Dict[str, Optional[pd.DataFrame]]:
            """Process all season data."""
            def safe_df(df: Optional[pd.DataFrame]) -> pd.DataFrame:
                return df if df is not None and not df.empty else pd.DataFrame()
        
            # Получаем все таблицы
            tables = safe_df(self.process_season_tables(season_id))
            metas = safe_df(self.process_season_meta(season_id))
            forms = safe_df(self.process_form_tables(season_id))
            summary = safe_df(self.process_league_summary(season_id))
            goals = safe_df(self.process_season_goals(season_id))
        
            # Повторяем мета/summary если есть таблицы
            n = max(len(tables), 1)
            meta_repeated = pd.concat([metas] * n, ignore_index=True) if not metas.empty else pd.DataFrame()
            summary_repeated = pd.concat([summary] * n, ignore_index=True) if not summary.empty else pd.DataFrame()
        
            # Собираем финальные таблицы
            if not tables.empty and not forms.empty:
                combined = pd.concat([
                    meta_repeated.reset_index(drop=True),
                    tables.reset_index(drop=True),
                    forms.reset_index(drop=True)
                ], axis=1)
            else:
                combined = pd.DataFrame()
        
            if not summary.empty and not goals.empty:
                combined1 = pd.concat([
                    summary_repeated.reset_index(drop=True),
                    goals.reset_index(drop=True)
                ], axis=1)
            else:
                combined1 = pd.DataFrame()
        
            return {
                'Table': combined,
                'Season_goals': combined1,
                # 'over_under': safe_df(self.process_over_under_stats(season_id)),
                # 'last_matches': safe_df(self.process_last_matches(season_id)),
                # и т.д.
            }
        def Leagues(self, frame2) -> Dict[str, Any]:
            """Full pipeline to process all data."""
            results = {}
            if frame2 is not None:
                if not frame2.empty and 'currentseason' in frame2.columns:
                    frame2 = frame2[~frame2['currentseason'].duplicated()].reset_index(drop=True)
                    for i, season_id in enumerate(frame2['currentseason']):
                        leag_season = f"{season_id}_{frame2['name'][i]}"
                        print('proceding  next season: ',leag_season)
                        results[leag_season] = self.process_season_data(season_id)
    
            return results


In [79]:
processor1 = SportsDataProcessor()
resultsth = processor1.Leagues(th['Австралия'][0])

proceding  next season:  120979_А-Лига
proceding  next season:  127557_НПЛ, Квинсленд
proceding  next season:  127883_НПЛ, Новый Южный Уэльс
proceding  next season:  128527_НПЛ, Западная Австралия
proceding  next season:  127721_НПЛ, Южная Австралия
proceding  next season:  127251_НПЛ, Виктория
proceding  next season:  129591_Капитал НПЛ 1
proceding  next season:  127885_НПЛ, Северный Новый Южный Уэльс
proceding  next season:  128529_НПЛ, Тасмания
proceding  next season:  127249_Victoria Premier League 1
proceding  next season:  119259_Кубок ФФА, Первый раунд, Штат СЮЗ
tablerows не список для сезона 119259. tablerows = None
⚠️ formrows пуст, возвращаю пустой DataFrame
proceding  next season:  127559_Queensland Premier League 1
proceding  next season:  123819_W-Лига
proceding  next season:  128541_Виктория Премьер-Лига, Женщины
proceding  next season:  127725_НПЛ, Южная Австралия, Женщины ПЛ, Финальный раунд
proceding  next season:  127561_Квинсленд Национальная Премьер-лига, Женщины


In [84]:
class Statistic_parser(SportsDataProcessor):
    def process_over_under_stats(self, season_id: int) -> Optional[pd.DataFrame]:
            """Process Over/Under statistics."""
            url = f"/stats_season_overunder/{season_id}"
            data = self.fetch_data(url)
            if not data:
                return None
    
            try:
                main_data = self._get_nested_data(data, ['doc', 0, 'data'])
                if not main_data:
                    return None
    
                stats = main_data.get('stats', {})
                team_stats = pd.concat(
                    [pd.json_normalize(stats[key]).assign(type=key)
                     for key in stats.keys()],
                    ignore_index=True
                )
    
                team_stats = team_stats.drop(columns=[
                    'level_1', 'team._doc', 'team.suffix', 'team.nickname'
                ], errors='ignore')
    
                league_totals = pd.json_normalize(main_data.get('league', {}).get('totals', {}))
                league_data = pd.concat([league_totals] * len(team_stats), ignore_index=True)
    
                return pd.concat([
                    team_stats.reset_index(drop=True),
                    league_data.add_prefix('league_')
                ], axis=1).drop(columns = ['homematches', 'awaymatches', 'team._rcid', 'team._sid', 'team.haslogo', 'team.website', 
                                'team.teamtypeid', 'type', 'league_matches', 'team.abbr'] ,axis=1, errors='ignore')
    
            except Exception as e:
                print(f"Over/Under stats error: {e}")
                return None
    def process_stat_data(self, season_id: int) -> Dict[str, Optional[pd.DataFrame]]:
            """Process all season data."""
            def safe_df(df: Optional[pd.DataFrame]) -> pd.DataFrame:
                return df if df is not None and not df.empty else pd.DataFrame()
        
            return {
            'over_under': self.process_over_under_stats(season_id),
            #'last_matches': self.process_last_matches(season_id),
            #'next_matches': self.process_next_matches(season_id),
            #'top_goals': self.process_top_goals(season_id),
            #'top_assists': self.process_top_assists(season_id),
            #'top_cards': self.process_top_cards(season_id),
            #'injuries': self.process_injuries(season_id),
            #'season_teamsstats': self.process_season_teamsstats(season_id),
            #'season_fixtures': self.process_season_fixtures(season_id),
            #'match_details': self.process_match_details(season_id),
            #'match_timelines': self.process_match_timelines(season_id),
            #'match_details_data': self.process_match_details_data(season_id),
            #'match_lineups': self.process_match_lineups(season_id),
            #'usual_lineup': self.process_usual_lineup(season_id),
            #'team_squad': self.process_team_squad(season_id),
            }
    def resultat(self, frame2) -> Dict[str, Any]:
            """Full pipeline to process all data."""
            results = {}
            if frame2 is not None:
                if not frame2.empty and 'currentseason' in frame2.columns:
                    frame2 = frame2[~frame2['currentseason'].duplicated()].reset_index(drop=True)
                    for i, season_id in enumerate(frame2['currentseason']):
                        leag_season = f"{season_id}_{frame2['name'][i]}"
                        print('proceding  next season: ',leag_season)
                        results[leag_season] = self.process_stat_data(season_id)
    
            return results

In [87]:
Static_proc = Statistic_parser()
resulting = Static_proc.resultat(th['Австралия'][0])

proceding  next season:  120979_А-Лига
proceding  next season:  127557_НПЛ, Квинсленд
proceding  next season:  127883_НПЛ, Новый Южный Уэльс
proceding  next season:  128527_НПЛ, Западная Австралия
proceding  next season:  127721_НПЛ, Южная Австралия
proceding  next season:  127251_НПЛ, Виктория
proceding  next season:  129591_Капитал НПЛ 1
proceding  next season:  127885_НПЛ, Северный Новый Южный Уэльс
proceding  next season:  128529_НПЛ, Тасмания
proceding  next season:  127249_Victoria Premier League 1
proceding  next season:  119259_Кубок ФФА, Первый раунд, Штат СЮЗ
proceding  next season:  127559_Queensland Premier League 1
proceding  next season:  123819_W-Лига
proceding  next season:  128541_Виктория Премьер-Лига, Женщины
proceding  next season:  127725_НПЛ, Южная Австралия, Женщины ПЛ, Финальный раунд
proceding  next season:  127561_Квинсленд Национальная Премьер-лига, Женщины


In [90]:
resulting['120979_А-Лига']['over_under']

,matches,team._id,team.name,team.mediumname,team.iscountry,team.sex,team.founded,winrate.p1.matches,winrate.p1.wins,winrate.p1.average,...,league_p2.3.5.over,league_p2.3.5.under,league_p2.4.5.totalover,league_p2.4.5.totalunder,league_p2.4.5.over,league_p2.4.5.under,league_p2.5.5.totalover,league_p2.5.5.totalunder,league_p2.5.5.over,league_p2.5.5.under
0,23,5966,Централ Кост,Централ Кост Маринерс ФК,False,m,2004,23,5,0.22,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
1,22,5970,Мельбурн Виктори,Мельбурн Виктори,False,m,2004,22,8,0.36,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
2,22,1156423,Auckland,Auckland FC,False,m,None,22,11,0.50,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
3,21,5968,Брисбен,Брисбен Роар ФК,False,m,2005,21,4,0.19,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
4,22,2934,Ньюкасл,Ньюкасл Юнайтед Джетс,False,m,2000,22,4,0.18,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
5,22,42210,Мельбурн Сити,Мельбурн Сити,False,m,2009,22,11,0.50,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
6,22,72926,Вестерн Сидней,Вестерн Сидней Уондерерс,False,m,2012,22,9,0.41,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
7,22,5971,Сидней,Сидней ФК,False,m,2004,22,5,0.23,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
8,22,7568,Веллингтон,Веллингтон Финикс ФК,False,m,2007,22,5,0.23,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
9,22,199320,Вестерн Юн,Вестерн Юнайтед,False,m,2017,22,8,0.36,...,0.12,0.88,4,140,0.03,0.97,0,144,0,1
